Problem Set 6: confidence intervals and hypothesis testing in multiple regression<br>
Jyoti Chandrawanshi<br>

In [2]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import math
import scipy.stats as stats

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_style('white')

C:\Users\goura\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
df = pd.read_csv("lead_mortality.csv")

In [4]:
# First we'll take a look at the data. Are there any null values that we have to worry about?
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   year           172 non-null    int64  
 1   city           172 non-null    object 
 2   state          172 non-null    object 
 3   age            172 non-null    float64
 4   hardness       172 non-null    int64  
 5   ph             172 non-null    float64
 6   infrate        172 non-null    float64
 7   typhoid_rate   172 non-null    float64
 8   np_tub_rate    172 non-null    float64
 9   mom_rate       172 non-null    float64
 10  population     172 non-null    float64
 11  precipitation  172 non-null    float64
 12  temperature    172 non-null    float64
 13  lead           172 non-null    int64  
 14  foreign_share  172 non-null    float64
dtypes: float64(10), int64(3), object(2)
memory usage: 20.3+ KB


#### 1. What is the average infant mortality rate (Infrate) for cities with lead pipes and for cities with nonleaded pipes. Is there a statistically significant difference in the averages? 

In [17]:
y_group1 = df[df["lead"] == 1]['infrate']
y_group1_mean = y_group1.mean()

y_group2 = df[df["lead"] == 0]['infrate']
y_group2_mean = y_group2.mean()
    
 
mean_diff = y_group1_mean - y_group2_mean  


print("The Infrate for group 1 is", round(y_group1_mean, 5), 
      ", the Infrate for group 2 is", round(y_group2_mean, 5),
     "and the difference is", round(mean_diff, 5))

The Infrate for group 1 is 0.40326 , the Infrate for group 2 is 0.38117 and the difference is 0.02209


In [18]:
se_group1 = stats.sem(y_group1)
se_group2 = stats.sem(y_group2)

print("The standard error for group 1 is", round(se_group1, 5), 
      "and the standard error for group 2 is", round(se_group2, 5))

se_diff = math.sqrt(se_group1**2 + se_group2**2)
#Hint: use math.sqrt for square roots, and use "**" for exponents

print("The standard error of the difference is", round(se_diff, 5))

The standard error for group 1 is 0.01415 and the standard error for group 2 is 0.01992
The standard error of the difference is 0.02444


t = 0.02209 / 0.02444<br>
t = .90384615


In [7]:
y_group1 = df[df["lead"] == 1]['infrate']
stats_numeric1 = y_group1.describe()
print (stats_numeric1)

count    117.000000
mean       0.403258
std        0.153087
min        0.141361
25%        0.295796
50%        0.390323
75%        0.478431
max        0.844466
Name: infrate, dtype: float64


In [8]:
y_group2 = df[df["lead"] == 0]['infrate']
stats_numeric2 = y_group2.describe()
print (stats_numeric2)

count    55.000000
mean      0.381168
std       0.147759
min       0.109756
25%       0.277626
50%       0.375000
75%       0.453295
max       0.751553
Name: infrate, dtype: float64


formula for t test is = m1 -m2 / sqrt(sd1^2/n1+sd2^2/n2) <br>
substitutes the value of mean and standard deviatiion <br>

t = 0.403258 - 0.381168  /sqrt(0.153087^2/117+ 0.147759^2/55) <br>
t = .02209 /sqrt(.000597) <br>
t = .02209/.02443<br>
t = .9042 <br>
Since the the t value is less than 1.96 so, we can not reject the null hypothesis or we can say that the average means are approximately equal.


#### 2. The amount of lead leached from lead pipes depends on the chemistry of the water running through the pipes. The more acidic the water (the lower its pH), the more lead is leeched.

#### a. Run a regression of Infrate on Lead, pH, and the interaction term Lead * pH 

In [10]:
df.lead = df.lead.astype(float)
df['lead_ph'] = df['lead']*df['ph']

X = df[['lead','ph','lead_ph']]
Y = df["infrate"]
X = sm.add_constant(X.values)
model = sm.OLS(Y, X).fit(cov_type = 'HC1')
model.summary(xname = ['const', 'lead','ph','lead_ph'])

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                infrate   R-squared:                       0.272
Model:                            OLS   Adj. R-squared:                  0.259
Method:                 Least Squares   F-statistic:                     20.97
Date:                Fri, 26 Feb 2021   Prob (F-statistic):           1.37e-11
Time:                        15:24:19   Log-Likelihood:                 108.52
No. Observations:                 172   AIC:                            -209.0
Df Residuals:                     168   BIC:                            -196.5
Df Model:                           3                                         
Covariance Type:                  HC1                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9189      0.150      6.106      0.000       0.624       1.214
lead           0.4618      0.208      2.224      0.026       0.055       0.869
ph            -0.0752      0.021     -3.588      0.000      -0.116      -0.034
lead_ph       -0.0569      0.028     -2.025      0.043      -0.112      -0.002
==============================================================================
Omnibus:                        4.916   Durbin-Watson:                   1.946
Prob(Omnibus):                  0.086   Jarque-Bera (JB):                4.987
Skew:                           0.411   Prob(JB):                       0.0826
Kurtosis:                       2.861   Cond. No.                         252.
==============================================================================

Warnings:
[1] Standard Errors are heteroscedasticity robust (HC1)
"""

#### b. The regression has four coefficients, including the intercept. Explain what each coefficient means. Does Lead have a statistically significant effect on infant mortality?

The mean change in ph by one unit will decrease infant mortality by 0.0752. when other variables are constant.The mean change in lead by one unit will change infant mortality by 0.4618. when other variables are constant. For mean change in ph * lead by one unit will decrease infant mortality by 0.0569 when other variables are constant. When ph and lead are not present or zero then the infant mortality is 0.9189. 

#### c. Does the effect of Lead on infant mortality depend on pH? Is this dependence statistically significant? 

From regression equation: <br>

infant mortality = 0.9189 + 0.4618 * lead - 0.0752	* ph - 0.0569 * lead_ph<br>

if mean value of lead will change by one unit then infant mortality will increase by 0.4618 when other variable are constant. for lead_ph, the mean value of lead_ph will change by one unit then infant mortality will decreses by 0.0569 when other variable are constant.Yes lead_ph has statistically significant effect on infant mortality because the p-value of lead_ph (0.043) is less than alpha at 0.05.

#### 3. The analysis in question 2 may suffer from omitted variable bias because it doesn't include other factors that affect infant mortality that may also be correlated with Lead and pH. When looking at other variables in the dataset, explain why you think the regression does or does not have omitted variable bias. 

In [ ]:
X = df[['lead','ph','age','hardness','typhoid_rate','np_tub_rate','mom_rate','population','precipitation','temperature','foreign_share']]
Y = df["infrate"]
X = sm.add_constant(X.values)
model = sm.OLS(Y, X).fit(cov_type = 'HC1')
model.summary(xname = ['const', 'lead','ph','age','hardness','typhoid_rate','np_tub_rate','mom_rate','population','precipitation','temperature','foreign_share'])